## xcube Data Store Framework - CCI Open Data Portal

*Please checkout the general introduction about xcube stores in the jupyter notebook xcube/examples/notebooks/store/Intro_xcube_store_framework.ipynb before jumping into this notebook :)* 

This notebook provides a walk-through demonstrating how to use xcube and the xcube Climate Change Initiative Open Data Portal (CCI) plugin to read and explore data from CCI Open Data Portal.


In order to run this notebook you need to install the xcube-cci plugin. You may do so by executing the following line in your terminal: 

```
$  conda install -c conda-forge xcube-cci
```

Or you can install xcube-cci from sources by following the instructions on https://github.com/dcs4cop/xcube-cci


In [1]:
# CCI Data Store requires asyncio. In order to work in Notebooks, we need to do the following:
from xcube.util.ipython import enable_asyncio
enable_asyncio()

In [2]:
# mandatory imports
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store

# Utilities for notebook visualization
from IPython.display import JSON
import shapely.geometry
import IPython.display
from IPython.display import JSON
import matplotlib.pyplot as plt

Configure matplotlib to display graphs inline directly in the notebook and set a sensible default figure size.

In [3]:
%matplotlib inline
plt.rcParams["figure.figsize"] = 16,8


Check wheather the `cciodp` store is among the available stores, if not please follow the installation information from the top of this notebook. 

In [4]:
JSON({e.name: e.metadata for e in find_data_store_extensions()})

<IPython.core.display.JSON object>

Usually we need more information to get the actual data store object. Which data store parameters are available for `cciodp`?

In [5]:
get_data_store_params_schema('cciodp')

Provide mandatory parameters to instantiate the store class:

In [6]:
store = new_data_store('cciodp')
store

Which datasets are provided? (the list may contain both gridded and vector datasets):

We may ask for a specific dataset ...

In [7]:
store.has_data('esacci.OC.5-days.L3S.CHLOR_A.multi-sensor.multi-platform.MERGED.3-1.geographic')

True

... but in many cases we want to query for certain criteria. How can we do that?

In [8]:
store.get_search_params_schema()

Now search, let's search for sea surface temperature data (SST) with a daily frequency: 

In [9]:
iterator = store.search_data(ecv='SST', frequency='day')
JSON([item.to_dict() for item in iterator])

<IPython.core.display.JSON object>

Which parameters must be passsed or are available to open the dataset? 

In [10]:
store.get_open_data_params_schema('esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.1-1.r1')

There are no required parameters, so we can decide what parameters we would like to provide them to open a dataset:

In [11]:
dataset = store.open_data('esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.1-1.r1', 
                          variable_names=['analysed_sst'],
                          time_range=['2008-07-01','2008-07-10'])

dataset

KeyError: 'variables'

Plot one time stamp of the dataset for a analysed_sst in order to take a brief look at the dataset:  

In [ ]:
dataset.analysed_sst.isel(time=1).plot.imshow(cmap='plasma')